In [2]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))

In [3]:
srFile = "examples/CoDec_Research/results/simulation_results/server/Set2_construal_vals_2025-05-11 15_02_12.469384.pickle"
with open(srFile, 'rb') as opn_file:
    construal_action_likelihoods = pickle.load(opn_file)


In [4]:
construal_action_likelihoods

{'dict_structure': '{scene_name: {construal_index: value}}',
 'tfrecord-00035-of-01000_85': {(0,): 1.0,
  (1,): 1.0,
  (2,): 1.0,
  (3,): 1.0,
  (4,): 0.975,
  (5,): 1.0,
  (35,): 1.0,
  (41,): 1.0,
  (0, 1, 2, 3, 4, 5, 35, 41): 1.0},
 'tfrecord-00046-of-01000_167': {(0,): 1.0,
  (1,): 1.0,
  (2,): 1.0,
  (3,): 1.0,
  (4,): 1.0,
  (5,): 1.0,
  (6,): 1.0,
  (7,): 1.0,
  (8,): 1.0,
  (10,): 1.0,
  (11,): 1.0,
  (12,): 1.0,
  (15,): 1.0,
  (17,): 1.0,
  (18,): 1.0,
  (19,): 1.0,
  (20,): 1.0,
  (24,): 1.0,
  (31,): 1.0,
  (0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 15, 17, 18, 19, 20, 24, 31): 0.975},
 'tfrecord-00047-of-01000_484': {(0,): 1.0,
  (2,): 0.975,
  (3,): 1.0,
  (4,): 1.0,
  (5,): 1.0,
  (6,): 1.0,
  (7,): 1.0,
  (8,): 1.0,
  (9,): 1.0,
  (10,): 1.0,
  (11,): 1.0,
  (12,): 1.0,
  (13,): 1.0,
  (14,): 1.0,
  (16,): 1.0,
  (17,): 1.0,
  (18,): 1.0,
  (19,): 1.0,
  (21,): 1.0,
  (22,): 1.0,
  (23,): 1.0,
  (24,): 1.0,
  (26,): 1.0,
  (27,): 0.975,
  (28,): 1.0,
  (29,): 1.0,
  (30,):